In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2

In [ ]:
image_size = [224, 224]


In [ ]:
inception = InceptionV3(input_shape=image_size + [3], weights='imagenet', include_top=False)


87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tomato_classes = []
individual_disease = []
image_paths = []
labels = []
d=0
for leaves in os.listdir('drive/MyDrive/Tomato/'):
    d=d+1
    for image in os.listdir('drive/MyDrive/Tomato/' + str(leaves)):
        individual_disease.append('drive/MyDrive/Tomato/' + str(leaves) + '/' + str(image))
        labels.append(leaves)
        path=('drive/MyDrive/Tomato/' + str(leaves) + '/' + str(image))
        img = cv2.imread(path)
        im =cv2.resize(img,(224,224))
        image_paths.append(im)
    tomato_classes.append(individual_disease)
    individual_disease = []
print("Labels:", labels)

Labels: ['Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus', 'Tomato_yellowLeaf_Curl_Virus',

In [ ]:
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(d, activation = 'softmax')(x)


In [ ]:
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [ ]:
data_arr = np.array(image_paths)
label_arr = np.array(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y= encoder.fit_transform(label_arr)
x= data_arr

In [ ]:

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.1, random_state=50)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
augmented_data_generator = train_datagen.flow(x_train, y_train, batch_size=50)

In [ ]:
len(augmented_data_generator)

64

In [ ]:
augmented_data_generator_1 = test_datagen.flow(x_test, y_test, batch_size=50)

In [ ]:
len(augmented_data_generator_1)

8

In [ ]:
x_test.shape

(351, 224, 224, 3)

In [26]:
mod = model.fit(
  augmented_data_generator,
  validation_data=(augmented_data_generator_1 ),
  verbose=1,
  epochs=10)


Epoch 1/10
64/64 [==============================] - 39s 603ms/step - loss: 0.6735 - accuracy: 0.8891 - val_loss: 2.6248 - val_accuracy: 0.7066
Epoch 2/10
64/64 [==============================] - 38s 588ms/step - loss: 0.7139 - accuracy: 0.8875 - val_loss: 2.4289 - val_accuracy: 0.7407
Epoch 3/10
64/64 [==============================] - 37s 566ms/step - loss: 0.6631 - accuracy: 0.8989 - val_loss: 1.7717 - val_accuracy: 0.8120
Epoch 4/10
64/64 [==============================] - 37s 567ms/step - loss: 0.6687 - accuracy: 0.8992 - val_loss: 2.5381 - val_accuracy: 0.7664
Epoch 5/10
64/64 [==============================] - 38s 596ms/step - loss: 0.7499 - accuracy: 0.8919 - val_loss: 2.1089 - val_accuracy: 0.8319
Epoch 6/10
64/64 [==============================] - 38s 596ms/step - loss: 0.4853 - accuracy: 0.9217 - val_loss: 1.8079 - val_accuracy: 0.8034
Epoch 7/10
64/64 [==============================] - 37s 580ms/step - loss: 0.5102 - accuracy: 0.9217 - val_loss: 1.6860 - val_accuracy: 0.8120

In [38]:
score = model.evaluate(augmented_data_generator_1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 110ms/step - loss: 1.8532 - accuracy: 0.8148
Test Loss: 1.853193759918213
Test accuracy: 0.8148148059844971


In [48]:
y_pred = model.predict(augmented_data_generator_1)

8/8 [==============================] - 1s 111ms/step


In [49]:
y_pred = np.argmax(y_pred, axis=1)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, labels=np.unique(y_test)))